In [ ]:
!pip install accelerate transformers diffusers

In [ ]:
import transformers

In [ ]:
from diffusers import DiffusionPipeline, LCMScheduler
import torch

model_id = "stabilityai/stable-diffusion-xl-base-1.0"
lcm_lora_id = "latent-consistency/lcm-lora-sdxl"

pipe = DiffusionPipeline.from_pretrained(model_id, variant="fp16")

pipe.load_lora_weights(lcm_lora_id)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.to(device="cuda", dtype=torch.float16)

prompt = "close-up photography of old man standing in the rain at night, in a street lit by lamps, leica 35mm summilux"
images = pipe(
    prompt=prompt,
    num_inference_steps=4,
    guidance_scale=1,
).images[0]

In [ ]:
images

In [ ]:
import time

start_time = time.perf_counter()

prompt = "close-up photography of man standing in the rain at night, in a street lit by lamps, leica 35mm summilux"
images = pipe(
    prompt=prompt,
    width=512,
    height=768,
    num_inference_steps=4,
    guidance_scale=1,
).images[0]

print(time.perf_counter() - start_time)

In [ ]:
images

In [ ]:
images = []
for num in range(8):
    generator = torch.Generator(device=pipe.device).manual_seed(num)
    image = pipe(
        prompt=prompt,
        num_inference_steps=4,
        guidance_scale=1,
        generator=generator,
    ).images[0]
    images.append(image)

In [ ]:
from IPython.display import display
for image in images:
    display(image)